In [1]:
import findspark
findspark.init()
import pyspark


In [2]:
sc = pyspark.SparkContext(appName="test_jupyterlab")

In [3]:
from pyspark import SQLContext

In [4]:
sql = SQLContext(sc)

In [5]:
base_url = "s3://omicidx.cancerdatasci.org/sra/NCBI_SRA_Mirroring_20180601_Full/"

In [6]:
def get_sra_dfs(base_url):
    """Read sra data from parquet (stored previously)
    
    :param: base_url the url of the parquet directory
    
    Returns: a dict with keys of entities ("experiment", "sample", etc.)
    """
    entities = "experiment sample run study".split()
    import os
    return dict((n, sql.read.parquet(os.path.join(base_url,"parquet","{}_parquet".format(n)))) for n in entities)

In [7]:
sra_dfs = get_sra_dfs(base_url)

In [8]:
def prepend_colnames(df, prepend, sep = "_"):
    for c in df.columns:
        if(c.startswith(prepend)):
            continue
        if(c.endswith('accession') & (c!='accession')):
            continue
        df = df.withColumnRenamed(c, prepend + sep + c)
    return(df)

In [9]:
e = prepend_colnames(sra_dfs['experiment'].drop('experiment_accession'), 'experiment')
r = prepend_colnames(sra_dfs['run'], 'run')
st = prepend_colnames(sra_dfs['study'].drop('study_accession'), 'study')
sa = prepend_colnames(sra_dfs['sample'], 'sample')

In [10]:
x=r.join(e, 'experiment_accession', 'left')\
   .join(st, 'study_accession', 'left')\
   .join(sa, 'sample_accession', 'left')\
   .cache()

In [11]:
y = x.limit(100000).repartition(25)

In [12]:
y.rdd.getNumPartitions()

25

In [43]:
y.write.json('s3://omicidx.cancerdatasci.org/tmp/full_json')

In [ ]:
# may need to change last line to: .save('sra_full2/doc', overwrite="true") 
# if the index exists and want to overwrite. Note that this will remove
# all index entries prior to writing.
x.write.format("org.elasticsearch.spark.sql")\
    .option("es.nodes.wan.only","true")\
    .option("es.net.http.auth.user","omicidx")\
    .option("es.net.http.auth.pass","Asdf1234%")\
    .option("es.net.ssl","true")\
    .option("es.nodes","c0b1083b4d78495d3b29a72d74fcea41.us-east-1.aws.found.io")\
    .option("es.port","9243")\
    .option("es.mapping.id", "run_accession")\
    .save('sra_full/doc')

In [32]:
y = x.sample(False, 0.01, 1).cache()

In [ ]:
sra_dfs['study'] = sra_dfs['study'].drop('study_accession')
sra_dfs['experiment'] = sra_dfs['experiment'].drop('experiment_accession')
for i in sra_dfs.keys():
    print(i)
    sra_dfs[i].write.format("org.elasticsearch.spark.sql")\
        .option("es.nodes.wan.only","true")\
        .option("es.net.http.auth.user","omicidx")\
        .option("es.net.http.auth.pass","Asdf1234%")\
        .option("es.net.ssl","true")\
        .option("es.nodes","c0b1083b4d78495d3b29a72d74fcea41.us-east-1.aws.found.io")\
        .option("es.port","9243")\
        .option("es.mapping.id", "accession")\
        .save('sra_{}/doc'.format(i), mode='overwrite')

run


In [21]:
sra_dfs['study'].columns

['BioProject',
 'GEO',
 'abstract',
 'accession',
 'alias',
 'attributes',
 'broker_name',
 'center_name',
 'description',
 'identifiers',
 'study_accession',
 'study_type',
 'title',
 'xrefs',
 'Status',
 'Received',
 'Published',
 'LastUpdate',
 'LastMetaUpdate',
 'ReplacedBy',
 'Insdc']

In [41]:
y = x.limit(50)

In [42]:
y.write.format("org.elasticsearch.spark.sql")\
    .option("es.nodes.wan.only","true")\
    .option("es.net.http.auth.user","omicidx")\
    .option("es.net.http.auth.pass","Asdf1234%")\
    .option("es.net.ssl","true")\
    .option("es.nodes","c0b1083b4d78495d3b29a72d74fcea41.us-east-1.aws.found.io")\
    .option("es.port","9243")\
    .option("es.mapping.id", "run_accession")\
    .save('sra_full/doc', mode='overwrite')

Py4JJavaError: An error occurred while calling o1058.save.
: java.lang.NullPointerException
	at org.elasticsearch.hadoop.rest.RestRepository.flush(RestRepository.java:196)
	at org.elasticsearch.hadoop.rest.RestRepository.delete(RestRepository.java:414)
	at org.elasticsearch.spark.sql.ElasticsearchRelation.insert(DefaultSource.scala:606)
	at org.elasticsearch.spark.sql.DefaultSource.createRelation(DefaultSource.scala:104)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:46)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:86)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(

In [38]:
y.show()

+----------------+---------------+--------------------+---------+--------------------+---------------+---------------+--------------------+----------+---------+-------------+----------+--------+---------------+---------+----------+----------+-------------------+-------------------+-------------------+-------------------+--------------+---------+--------------------+------------+--------------------+-------------------+----------------+---------------------+----------------------+----------------------+----------------------+--------------------+----------------------+---------------------------+----------------------------------------+-------------------------+--------------------------------+-------------------------------------+------------------------------+-----------------------+----------------------------+-------------------------+---------------------------+-------------------+--------------------+----------------+-----------------+-------------------+--------------------+-----

In [ ]:
df3 = sql.read.format("org.elasticsearch.spark.sql")\
    .option("es.nodes.wan.only","true")\
    .option("es.net.http.auth.user","")\
    .option("es.net.http.auth.pass","Asdf1234%")\
    .option("es.net.ssl","true")\
    .option("es.nodes","c0b1083b4d78495d3b29a72d74fcea41.us-east-1.aws.found.io")\
    .option("es.read.field.as.array.include","identifiers,attributes,xrefs")\
    .option("es.mapping.date.rich", "false")\
    .option("es.port","9200").load('sra_study/doc')